<a href="https://colab.research.google.com/github/mgersins-design/Machine-Learning/blob/main/HW6/FCNN_CIFAR_10_large.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [6]:
transform_process = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465],
        std=[0.2470, 0.2435, 0.2616]
    )
])

data_path = './data'

cifar10_train = datasets.CIFAR10(
    data_path, train=True, download=True, transform=transform_process
)
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True, transform=transform_process
)

batch_size = 64
train_loader = DataLoader(cifar10_train, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(cifar10_val, batch_size=batch_size, shuffle=False)

print(f"Trainingsdaten: {len(cifar10_train)} Bilder")
print(f"Validierungsdaten: {len(cifar10_val)} Bilder")

Trainingsdaten: 50000 Bilder
Validierungsdaten: 10000 Bilder


In [7]:
from torch.nn.modules.activation import LogSoftmax
import torch
import torch.nn as nn

model = nn.Sequential(
    nn.Linear(3 * 32 * 32, 1024),
    nn.ReLU(),
    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.Linear(512, 128),
    nn.ReLU(),
    nn.Linear(128, 10)
)

learning_rate = 1e-2
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss ()
n_epochs = 300

In [8]:
import time
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epochs in range(n_epochs):
  epoch_start_time = time.time()

  # --- Training Phase ---
  model.train()
  for imgs, labels in train_loader:

    imgs = imgs.to(device)
    labels = labels.to(device)

    batch_size = imgs.shape[0]
    out = model(imgs.view(batch_size, -1))
    loss = loss_fn(out, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  # --- Validation Phase ---
  model.eval()
  correct = 0
  total = 0

  with torch.no_grad():
    for imgs, labels in val_loader:
      imgs = imgs.to(device)
      labels = labels.to(device)

      outputs = model(imgs.view(imgs.shape[0], -1))
      _, predicted = torch.max(outputs, dim=1)
      total += labels.shape[0]
      correct += int((predicted == labels).sum())

      accuracy = correct/total

  epoch_end_time = time.time()
  epoch_time = epoch_end_time - epoch_start_time

  print(f"Epoch: {epochs+1:2d}, Time: {epoch_time:.2f}s, Training Loss: {loss.item():.4f}, Validation Accuracy: {accuracy:.4f}")

Epoch:  1, Time: 14.79s, Training Loss: 1.3819, Validation Accuracy: 0.3609
Epoch:  2, Time: 14.78s, Training Loss: 1.8954, Validation Accuracy: 0.4287
Epoch:  3, Time: 14.77s, Training Loss: 1.1645, Validation Accuracy: 0.4582
Epoch:  4, Time: 14.75s, Training Loss: 1.6510, Validation Accuracy: 0.4781
Epoch:  5, Time: 14.81s, Training Loss: 1.2610, Validation Accuracy: 0.4891
Epoch:  6, Time: 14.58s, Training Loss: 0.9112, Validation Accuracy: 0.5030
Epoch:  7, Time: 14.62s, Training Loss: 0.7051, Validation Accuracy: 0.5187
Epoch:  8, Time: 14.63s, Training Loss: 1.2946, Validation Accuracy: 0.5013
Epoch:  9, Time: 14.68s, Training Loss: 1.3300, Validation Accuracy: 0.5187
Epoch: 10, Time: 14.77s, Training Loss: 1.0933, Validation Accuracy: 0.4941
Epoch: 11, Time: 14.61s, Training Loss: 0.7249, Validation Accuracy: 0.5400
Epoch: 12, Time: 14.60s, Training Loss: 1.3446, Validation Accuracy: 0.4847
Epoch: 13, Time: 14.64s, Training Loss: 0.8771, Validation Accuracy: 0.5197
Epoch: 14, T